Super Smash Brothers Melee (SSBM) is a fighting game released by Nintendo for the Nintendo Gamecube in 2001. Despite its age, SSBM has maintained an active competitive scene throughout most of its lifetime, with recent years seeing some of the highest attendance numbers for tournaments in its competitive history (this can be attributed to a multitude of factors, such as the release of "The Smash Brothers" Documentary and the game's inclusion into the EVO tournament series, the largest annual fighting game tournament in the world). Competitive SSBM tournaments are double elimination brackets with the majority of "sets" being best of three or best of five. Each game in a set consists of two players (who each can pick a character from a roster of 26 possible characteres) each with four lives ("stocks"), who can pick a stage to play on from a specific set of possibles stages. The use of items and other game-influencing mechanics is prohibited throughout the tournament.

Until recently, the majority of statistics related to competitive SSBM have been "high-level", i.e they hasve mostly involved studying head-to-head matchups of players, winrates of specific players, and winrates of specific characters. While there is much that can be learned from studying these statistics, there is a lack of analysis of in-game statistics. This is analogous to a lack of statistics such as RBI and Batting Average for competitive Baseball analysis. 

Recent technology - specifically the SmashSight project, which utilizes neural networks to analyze footage of competitive SSBM gameplay in a frame-by-frame manner - allows for the aforementioned in-game analysis. SmashSight specifically takes in footage of competitive SSBM gameplay and returns a comma seperated table containing frame-by-frame radsings of each player's stock count and "percentage" points (which is a rough measure of their health - the higher a player's percentage, the further they are launched by getting hit). 

In this project we utilize SmashSight's publicly available data of the majority of the top eight sets from the Genesis 5 tournament. A main topic we explore is how winrate is affected via intermediate statistics that we gather from the data (such as average percent dealt per frame). 

In [3]:
import pandas as pd
import requests

In [43]:
#These are helper functions to make the data collection process easier
def standardize_name(char):
    if 'Fox' in char:
        return 'Fox'
    elif 'Sheik' in char:
        return 'Shiek'
    elif 'JigglyPuff' in char:
        return 'JigglyPuff'
    elif 'Peach' in char:
        return 'Peach'
    elif 'Falco' in char:
        return 'Falco'
    elif 'Pikachu' in char:
        return 'Pikachu'
    elif 'Samus' in char:
        return 'Samus'
    else: #should never be reached with this dataset
        return ''

def get_set_winner(list_of_game_data):
    return ''
    #TODO: returns winner of a set, as given by the list of dfs
    
#Reads in a url for the SmashSight data and parses it
#Returns a dictionary containing the player names, the player chars, the stage, the winner, and a dataframe containing the
#frame-by-frame data for the game
def read_fbf_data(url):
    data = requests.get(url).text.split('\n')
    del data[0]
    del data[1]
    rows = []
    cols = ['frame', 'L_Player_dmg', 'R_Player_dmg', 'L_Player_stocks', 'R_Player_stocks']
    meta_data = data[0].split(',')
    meta_data = [e for e in meta_data if e != 'NA']
    ret = {}
    L_Player_name = meta_data[1]
    R_Player_name = meta_data[2]
    L_Player_char = standardize_name(meta_data[3])
    R_Player_char = standardize_name(meta_data[4])
    Stage = meta_data[5]
    Winner = ''
    if meta_data[3] in meta_data[6] :
        Winner = L_Player_name
    else:
        Winner = R_Player_name
    ret['L_Player_Name'] = L_Player_name
    ret['R_Player_Name'] = R_Player_name
    ret['L_Player_Char'] = L_Player_char
    ret['R_Player_Char'] = R_Player_char
    ret['Stage'] = Stage
    ret['Winner'] = Winner
    
    for i in range(1, len(data)):
        row = []
        frame_data = data[i].split(',')
        frame_data = [e for e in frame_data if e !='NA']
        frame = frame_data[0]
        L_Player_dmg = frame_data[1]
        R_Player_dmg = frame_data[2]
        L_Player_stocks = frame_data[3]
        R_Player_stocks = frame_data[4]
        row = [frame, L_Player_dmg, R_Player_dmg, L_Player_stocks, R_Player_stocks]
        rows.append(row)
        if url == 'https://smashsight.com/wp-content/uploads/2018/11/genesis-5_axe-VS-mango_4.txt' and frame == '4627':
            break
    df_ret = pd.DataFrame(data=rows, columns = cols)
    ret['game_data'] = df_ret
    return ret

def convert_to_int(arr):
    ret = []
    for ele in arr:
        ret.append(int(ele))
        
    return ret

#returns the frames on which a given player died 
#i.e if at frame 461 the player has 4 stocks and at frame 462 the player has 3 stocks,
#the number 462 will be returned (possibly among others)
def detect_death(frames, stock_count):
    ret = []
    for fr in range(1, len(frames)):
        if(stock_count[fr] < stock_count[fr - 1]):
            ret.append(fr)
    return ret

#returns the death percentages for a given player for a given game
def find_death_pct(frames, pcts, stocks):
    ret = []
    death_frames = detect_death(frames, stocks)
    for fr in death_frames:
        ret.append(pcts[fr - 1])
    return ret
    
#computes the average death percentages for a given player on a given game
def compute_avg_death_pct(frames, pcts, stocks):
    avg = 0
    death_pcts = find_death_pct(frames, pcts, stocks)
    for pct in death_pcts:
        avg = avg + pct
    
    if(len(death_pcts) != 4):
        avg = avg/(len(death_pcts) + 1)
    else:
        avg = avg/(len(death_pcts))
        
    return avg
       
#computes the average dmg taken by a player over a given frame range
def compute_avg_dmg_taken(frame, pcts, start, end):
    avg = 0
    #frames of invulnerability
    angel_frames = 0
    for fr in range(start, end - 1):
        if(pcts[fr] == -1):
            angel_frames = angel_frames + 1
            continue
        #damage differential per frame
        #will always be nonnegative
        diff = pcts[fr + 1] - pcts[fr]
        avg = avg + diff
    #average over the number of frames in the range
    avg = avg/(end - start - angel_frames)
    return avg

def compute_avg_dmg_taken_per_stock(frames, pcts, stocks):
    death_frames = detect_death(frames, stocks)
    avg = 0
    
    #used to make the computations correct
    death_frames.insert(0, 0)
    if len(death_frames) != 4:
        death_frames.append(frames[len(frames) - 1])
        
    for i in range(1, len(death_frames) - 1):
        if i == 0:
            avg = avg + compute_avg_dmg_taken(frames, pcts, 0, death_frames[i])
        else:
            avg = avg + compute_avg_dmg_taken(frames, pcts, death_frames[i - 1], death_frames[i])
        i = i + 1
    avg = avg/(len(death_frames))
    return avg
    
#computes the stock efficiency of the subject player, defined as avg_death_pct/avg_kill_pct
def compute_stock_score(frames, tar_pcts, tar_stocks, sub_pcts, sub_stocks):
    avg_kill_pct = compute_avg_death_pct(frames, tar_pcts, tar_stocks)
    avg_death_pct = compute_avg_death_pct(frames, sub_pcts, sub_stocks)
    return (avg_death_pct/avg_kill_pct)

def compute_dmg_score(frames, tar_pcts, tar_stocks, sub_pcts, sub_stocks):
    avg_dmg_dealt = compute_avg_dmg_taken_per_stock(frames, tar_pcts, tar_stocks)
    avg_dmg_taken = compute_avg_dmg_taken_per_stock(frames, sub_pcts, sub_stocks)
    return (avg_dmg_dealt/avg_dmg_taken)
    
    
def killpct_R(row):
    if row['detect_death_L'] == 1:
        return int(row['L_Player_dmg'])
    else:
        return 0
    
def deathpct_L(row):
    if row['detect_death_L'] == 1:
        return int(row['L_Player_dmg'])
    else:
        return 0
    
def killpct_L(row):
    if row['detect_death_R'] == 1:
        return int(row['R_Player_dmg'])
    else:
        return 0
    
def stockeff(row):
    if row['avg_kill_pct'] == 0:
        return row['avg_death_pct']/300
    else:
        return row['avg_death_pct']/row['avg_kill_pct']

def win_L(row):
    if row['L_Player_Name'] == row['Winner']:
        return True
    else:
        return False

def win_R(row):
    if row['R_Player_Name'] == row['Winner']:
        return True
    else:
        return False

In [11]:
# Gathering data from SmashSight CSVs, collecting the resultant dataframes into a single dictionary 
#(one dataframe per game)
game_fbf_data = {}
player_data = {}
game_fbf_data['genesis 5_armada VS leffen_1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-leffen_1.txt')
game_fbf_data['genesis 5_armada VS leffen_2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-leffen_2.txt')
game_fbf_data['genesis 5_armada VS leffen_3'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-leffen_3.txt')
game_fbf_data['genesis 5_armada VS leffen_4'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-leffen_4.txt')
game_fbf_data['genesis 5_armada VS plup_1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-plup_1.txt')
game_fbf_data['genesis 5_armada VS plup_2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-plup_2.txt')
game_fbf_data['genesis 5_armada VS plup_3'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-plup_3.txt')
game_fbf_data['genesis 5_leffen VS hbox_1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_leffen-VS-hbox_1.txt')
game_fbf_data['genesis 5_leffen VS hbox_2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_leffen-VS-hbox_2.txt')
game_fbf_data['genesis 5_leffen VS hbox_3'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_leffen-VS-hbox_3.txt')
game_fbf_data['genesis 5_leffen VS hbox_4'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_leffen-VS-hbox_4.txt')
game_fbf_data['genesis 5_leffen VS hbox_5'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_leffen-VS-hbox_5.txt')
game_fbf_data['genesis 5_armada VS lucky_1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-lucky_1.txt')
game_fbf_data['genesis 5_armada VS lucky_2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-lucky_2.txt')
game_fbf_data['genesis 5_armada VS lucky_3'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_armada-VS-lucky_3.txt')
game_fbf_data['genesis 5_hugs VS lucky_1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hugs-VS-lucky_1.txt')
game_fbf_data['genesis 5_hugs VS lucky_2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hugs-VS-lucky_2.txt')
game_fbf_data['genesis 5_hugs VS lucky_3'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hugs-VS-lucky_3.txt')
game_fbf_data['genesis 5_axe VS mango_1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_axe-VS-mango_1.txt')
game_fbf_data['genesis 5_axe VS mango_2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_axe-VS-mango_2.txt')
game_fbf_data['genesis 5_axe VS mango_3'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_axe-VS-mango_3.txt')
game_fbf_data['genesis 5_axe VS mango_4'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_axe-VS-mango_4.txt')
game_fbf_data['genesis 5_axe VS mango_5'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_axe-VS-mango_5.txt')
game_fbf_data['genesis 5_plup VS hbox_1_WF'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_plup-VS-hbox_1_WF.txt')
game_fbf_data['genesis 5_plup VS hbox_2_WF'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_plup-VS-hbox_2_WF.txt')
game_fbf_data['genesis 5_plup VS hbox_3_WF'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_plup-VS-hbox_3_WF.txt')
game_fbf_data['genesis 5_plup VS hbox_4_WF'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_plup-VS-hbox_4_WF.txt')
game_fbf_data['genesis 5_hbox VS plup_1_GF1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hbox-VS-plup_1_GF1.txt')
game_fbf_data['genesis 5_hbox VS plup_2_GF1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hbox-VS-plup_2_GF1.txt')
game_fbf_data['genesis 5_hbox VS plup_3_GF1'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hbox-VS-plup_3_GF1.txt')
game_fbf_data['genesis 5_hbox VS plup_1_GF2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hbox-VS-plup_1_GF2.txt')
game_fbf_data['genesis 5_hbox VS plup_2_GF2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hbox-VS-plup_2_GF2.txt')
game_fbf_data['genesis 5_hbox VS plup_3_GF2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hbox-VS-plup_3_GF2.txt')
game_fbf_data['genesis 5_hbox VS plup_4_GF2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hbox-VS-plup_4_GF2.txt')
game_fbf_data['genesis 5_hbox VS plup_5_GF2'] = read_fbf_data('https://smashsight.com/wp-content/uploads/2018/11/genesis-5_hbox-VS-plup_5_GF2.txt')

In [52]:

for k in game_fbf_data:
    player_data[game_fbf_data[k]['L_Player_Name']] = {}
    player_data[game_fbf_data[k]['R_Player_Name']] = {}

cols = ['Versus', 'Avg_Death_Pct', 'Avg_Kill_Pct', 'Avg_Dmg_Dealt_Per_Frame', 'Avg_Dmg_Taken_Per_Frame', 'Stock_Score', 'Dmg_Score', 'Player_Char', 'Versus_Char', 'Stage', 'Won']
for player in player_data:
    rows = []
    for game in game_fbf_data:
        if player == game_fbf_data[game]['L_Player_Name']:
            frames = convert_to_int(game_fbf_data[game]['game_data']['frame'])
            r_player_pcts = convert_to_int(game_fbf_data[game]['game_data']['R_Player_dmg'])
            r_player_stocks = convert_to_int(game_fbf_data[game]['game_data']['R_Player_stocks'])
            l_player_pcts = convert_to_int(game_fbf_data[game]['game_data']['L_Player_dmg'])
            l_player_stocks = convert_to_int(game_fbf_data[game]['game_data']['L_Player_stocks'])
            l_player_char = game_fbf_data[game]['L_Player_Char']
            r_player_char = game_fbf_data[game]['R_Player_Char']
            
            Won = (player == game_fbf_data[game]['Winner'])
            Versus = game_fbf_data[game]['R_Player_Name']
            Avg_Death_Pct = compute_avg_death_pct(frames, l_player_pcts, l_player_stocks)
            Avg_Kill_Pct = compute_avg_death_pct(frames, r_player_pcts, r_player_stocks)
            Stage = game_fbf_data[game]['Stage']
            Avg_Dmg_Dealt_Per_Frame = compute_avg_dmg_taken_per_stock(frames, r_player_pcts, r_player_stocks)
            Avg_Dmg_Taken_Per_Frame = compute_avg_dmg_taken_per_stock(frames, l_player_pcts, l_player_stocks)
            Stock_Score = compute_stock_score(frames, r_player_pcts, r_player_stocks, l_player_stocks, l_player_stocks)
            Dmg_Score = compute_dmg_score(frames, r_player_pcts, r_player_stocks, l_player_pcts, l_player_stocks)
            rows.append([Versus, Avg_Death_Pct, Avg_Kill_Pct, Avg_Dmg_Dealt_Per_Frame, Avg_Dmg_Taken_Per_Frame, Stock_Score, Dmg_Score, l_player_char, r_player_char, Stage, Won])
            
        elif player == game_fbf_data[game]['R_Player_Name']:
            frames = convert_to_int(game_fbf_data[game]['game_data']['frame'])
            r_player_pcts = convert_to_int(game_fbf_data[game]['game_data']['R_Player_dmg'])
            r_player_stocks = convert_to_int(game_fbf_data[game]['game_data']['R_Player_stocks'])
            l_player_pcts = convert_to_int(game_fbf_data[game]['game_data']['L_Player_dmg'])
            l_player_stocks = convert_to_int(game_fbf_data[game]['game_data']['L_Player_stocks'])
            l_player_char = game_fbf_data[game]['L_Player_Char']
            r_player_char = game_fbf_data[game]['R_Player_Char']
            
            Won = (player == game_fbf_data[game]['Winner'])
            Versus = game_fbf_data[game]['L_Player_Name']
            Avg_Death_Pct = compute_avg_death_pct(frames, r_player_pcts, r_player_stocks)
            Avg_Kill_Pct = compute_avg_death_pct(frames, l_player_pcts, l_player_stocks)
            Stage = game_fbf_data[game]['Stage']
            Avg_Dmg_Dealt_Per_Frame = compute_avg_dmg_taken_per_stock(frames, l_player_pcts, l_player_stocks)
            Avg_Dmg_Taken_Per_Grame = compute_avg_dmg_taken_per_stock(frames, r_player_pcts, r_player_stocks)
            Stock_Score = compute_stock_score(frames, l_player_pcts, l_player_stocks, r_player_pcts, r_player_stocks)
            Dmg_Score = compute_dmg_score(frames, l_player_pcts, l_player_stocks, r_player_pcts, r_player_stocks)
            rows.append([Versus, Avg_Death_Pct, Avg_Kill_Pct, Avg_Dmg_Dealt_Per_Frame, Avg_Dmg_Taken_Per_Frame, Stock_Score, Dmg_Score, r_player_char, l_player_char, Stage, Won])
            
        else:
            continue
    df = pd.DataFrame(data=rows, columns=cols)
    player_data[player] = df

char_data = {}
for k in game_fbf_data:
    char_data[game_fbf_data[k]['L_Player_Char']] = {}
    char_data[game_fbf_data[k]['R_Player_Char']] = {}
    
#print(char_data) 
#cols = ['Versus', 'Avg_Death_Pct', 'Avg_Kill_Pct', 'Avg_Dmg_Dealt_Per_Frame', 'Avg_Dmg_Taken_Per_Frame', 'Stock_Eff', 'Dmg_Eff', 'Stage', 'Won']
#for char in char_data:
#    rows = []
#    for player, player_table in player_data.iteritems():
#        if char == player_table['Player_Char']:
#            Versus = player_table['Versus_Char']
#            Avg_Death_Pct = player_table[]
#        else:
#            continue
    
    
    
    

In [47]:
pd.options.display.max_rows = 5000

game_fbf_data['genesis 5_armada VS leffen_1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS leffen_1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS leffen_1'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS leffen_1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS leffen_1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS leffen_1'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS leffen_1']['death_pct_L'] = game_fbf_data['genesis 5_armada VS leffen_1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_1']['death_pct_R'] = game_fbf_data['genesis 5_armada VS leffen_1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_1']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS leffen_1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_1']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS leffen_1'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_1']['Win_L'] = game_fbf_data['genesis 5_armada VS leffen_1'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_1']['Win_R'] = game_fbf_data['genesis 5_armada VS leffen_1'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS leffen_2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS leffen_2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS leffen_2'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS leffen_2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS leffen_2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS leffen_2'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS leffen_2']['death_pct_L'] = game_fbf_data['genesis 5_armada VS leffen_2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_2']['death_pct_R'] = game_fbf_data['genesis 5_armada VS leffen_2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_2']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS leffen_2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_2']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS leffen_2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_2']['Win_L'] = game_fbf_data['genesis 5_armada VS leffen_2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_2']['Win_R'] = game_fbf_data['genesis 5_armada VS leffen_2'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS leffen_3']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS leffen_3'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS leffen_3'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS leffen_3']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS leffen_3'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS leffen_3'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS leffen_3']['death_pct_L'] = game_fbf_data['genesis 5_armada VS leffen_3'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_3']['death_pct_R'] = game_fbf_data['genesis 5_armada VS leffen_3'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_3']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS leffen_3'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_3']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS leffen_3'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_3']['Win_L'] = game_fbf_data['genesis 5_armada VS leffen_3'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_3']['Win_R'] = game_fbf_data['genesis 5_armada VS leffen_3'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS leffen_4']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS leffen_4'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS leffen_4'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS leffen_4']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS leffen_4'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS leffen_4'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS leffen_4']['death_pct_L'] = game_fbf_data['genesis 5_armada VS leffen_4'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_4']['death_pct_R'] = game_fbf_data['genesis 5_armada VS leffen_4'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_4']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS leffen_4'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_4']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS leffen_4'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_4']['Win_L'] = game_fbf_data['genesis 5_armada VS leffen_4'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS leffen_4']['Win_R'] = game_fbf_data['genesis 5_armada VS leffen_4'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS plup_1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS plup_1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS plup_1'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS plup_1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS plup_1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS plup_1'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS plup_1']['death_pct_L'] = game_fbf_data['genesis 5_armada VS plup_1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_1']['death_pct_R'] = game_fbf_data['genesis 5_armada VS plup_1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_1']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS plup_1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_1']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS plup_1'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_1']['Win_L'] = game_fbf_data['genesis 5_armada VS plup_1'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_1']['Win_R'] = game_fbf_data['genesis 5_armada VS plup_1'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS plup_2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS plup_1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS plup_2'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS plup_2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS plup_1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS plup_2'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS plup_2']['death_pct_L'] = game_fbf_data['genesis 5_armada VS plup_2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_2']['death_pct_R'] = game_fbf_data['genesis 5_armada VS plup_2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_2']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS plup_2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_2']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS plup_2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_2']['Win_L'] = game_fbf_data['genesis 5_armada VS plup_2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_2']['Win_R'] = game_fbf_data['genesis 5_armada VS plup_2'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS plup_3']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS plup_3'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS plup_3'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS plup_3']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS plup_3'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS plup_3'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS plup_3']['death_pct_L'] = game_fbf_data['genesis 5_armada VS plup_3'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_3']['death_pct_R'] = game_fbf_data['genesis 5_armada VS plup_3'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_3']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS plup_3'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_3']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS plup_3'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_3']['Win_L'] = game_fbf_data['genesis 5_armada VS plup_3'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS plup_3']['Win_R'] = game_fbf_data['genesis 5_armada VS plup_3'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS lucky_1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS lucky_1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS lucky_1'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS lucky_1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS lucky_1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS lucky_1'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS lucky_1']['death_pct_L'] = game_fbf_data['genesis 5_armada VS lucky_1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_1']['death_pct_R'] = game_fbf_data['genesis 5_armada VS lucky_1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_1']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS lucky_1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_1']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS lucky_1'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_1']['Win_L'] = game_fbf_data['genesis 5_armada VS lucky_1'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_1']['Win_R'] = game_fbf_data['genesis 5_armada VS lucky_1'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS lucky_2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS lucky_2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS lucky_2'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS lucky_2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS lucky_2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS lucky_2'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS lucky_2']['death_pct_L'] = game_fbf_data['genesis 5_armada VS lucky_2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_2']['death_pct_R'] = game_fbf_data['genesis 5_armada VS lucky_2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_2']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS lucky_2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_2']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS lucky_2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_2']['Win_L'] = game_fbf_data['genesis 5_armada VS lucky_2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_2']['Win_R'] = game_fbf_data['genesis 5_armada VS lucky_2'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_armada VS lucky_3']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_armada VS lucky_3'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS lucky_3'].R_Player_stocks))
game_fbf_data['genesis 5_armada VS lucky_3']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_armada VS lucky_3'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_armada VS lucky_3'].L_Player_stocks))
game_fbf_data['genesis 5_armada VS lucky_3']['death_pct_L'] = game_fbf_data['genesis 5_armada VS lucky_3'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_3']['death_pct_R'] = game_fbf_data['genesis 5_armada VS lucky_3'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_3']['kill_pct_L'] = game_fbf_data['genesis 5_armada VS lucky_3'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_3']['kill_pct_R'] = game_fbf_data['genesis 5_armada VS lucky_3'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_3']['Win_L'] = game_fbf_data['genesis 5_armada VS lucky_3'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_armada VS lucky_3']['Win_R'] = game_fbf_data['genesis 5_armada VS lucky_3'].apply(lambda row: win_R(row),axis=1)



Armada = pd.DataFrame([['arm v lef 1', game_fbf_data['genesis 5_armada VS leffen_1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS leffen_1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS leffen_1']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS leffen_1']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS leffen_1']['Win_L'][0]]],columns=['game','avg_kill_pct','avg_death_pct','Win'])

Armada.loc[1] = ['arm v lef 2', game_fbf_data['genesis 5_armada VS leffen_2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS leffen_2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS leffen_2']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS leffen_2']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS leffen_2']['Win_L'][0]]

Armada.loc[2] = ['arm v lef 3', game_fbf_data['genesis 5_armada VS leffen_3']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS leffen_3']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS leffen_3']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS leffen_3']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS leffen_3']['Win_L'][0]]

Armada.loc[3] = ['arm v lef 4', game_fbf_data['genesis 5_armada VS leffen_4']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS leffen_4']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS leffen_4']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS leffen_4']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS leffen_4']['Win_L'][0]]

Armada.loc[4] = ['arm v plup 1', game_fbf_data['genesis 5_armada VS plup_1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS plup_1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS plup_1']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS plup_1']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS plup_1']['Win_L'][0]]

Armada.loc[5] = ['arm v plup 2', game_fbf_data['genesis 5_armada VS plup_2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS plup_2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS plup_2']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS plup_2']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS plup_2']['Win_L'][0]]

Armada.loc[6] = ['arm v plup 3', game_fbf_data['genesis 5_armada VS plup_3']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS plup_3']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS plup_3']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS plup_3']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS plup_3']['Win_L'][0]]

Armada.loc[7] = ['arm v luck 1', game_fbf_data['genesis 5_armada VS lucky_1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS lucky_1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS lucky_1']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS lucky_1']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS lucky_1']['Win_L'][0]]

Armada.loc[8] = ['arm v luck 2', game_fbf_data['genesis 5_armada VS lucky_2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS lucky_2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS lucky_2']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS lucky_2']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS lucky_2']['Win_L'][0]]

Armada.loc[9] = ['arm v luck 3', game_fbf_data['genesis 5_armada VS lucky_3']['kill_pct_L'].sum()/game_fbf_data['genesis 5_armada VS lucky_3']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_armada VS lucky_3']['death_pct_L'].sum()/game_fbf_data['genesis 5_armada VS lucky_3']['detect_death_L'].sum(),game_fbf_data['genesis 5_armada VS lucky_3']['Win_L'][0]]

Armada['avgd_stock_efficiency'] = Armada.apply(lambda row: stockeff(row),axis=1)
print(Armada)

game_fbf_data['genesis 5_leffen VS hbox_1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_1'].R_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_1'].L_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_1']['death_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_1']['death_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_1']['kill_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_1']['kill_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_1'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_1']['Win_L'] = game_fbf_data['genesis 5_leffen VS hbox_1'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_1']['Win_R'] = game_fbf_data['genesis 5_leffen VS hbox_1'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_leffen VS hbox_2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_2'].R_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_2'].L_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_2']['death_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_2']['death_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_2']['kill_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_2']['kill_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_2']['Win_L'] = game_fbf_data['genesis 5_leffen VS hbox_2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_2']['Win_R'] = game_fbf_data['genesis 5_leffen VS hbox_2'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_leffen VS hbox_3']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_3'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_3'].R_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_3']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_3'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_3'].L_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_3']['death_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_3'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_3']['death_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_3'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_3']['kill_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_3'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_3']['kill_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_3'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_3']['Win_L'] = game_fbf_data['genesis 5_leffen VS hbox_3'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_3']['Win_R'] = game_fbf_data['genesis 5_leffen VS hbox_3'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_leffen VS hbox_4']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_4'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_4'].R_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_4']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_4'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_4'].L_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_4']['death_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_4'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_4']['death_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_4'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_4']['kill_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_4'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_4']['kill_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_4'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_4']['Win_L'] = game_fbf_data['genesis 5_leffen VS hbox_4'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_4']['Win_R'] = game_fbf_data['genesis 5_leffen VS hbox_4'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_leffen VS hbox_5']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_5'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_5'].R_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_5']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_leffen VS hbox_5'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_leffen VS hbox_5'].L_Player_stocks))
game_fbf_data['genesis 5_leffen VS hbox_5']['death_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_5'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_5']['death_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_5'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_5']['kill_pct_L'] = game_fbf_data['genesis 5_leffen VS hbox_5'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_5']['kill_pct_R'] = game_fbf_data['genesis 5_leffen VS hbox_5'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_5']['Win_L'] = game_fbf_data['genesis 5_leffen VS hbox_5'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_leffen VS hbox_5']['Win_R'] = game_fbf_data['genesis 5_leffen VS hbox_5'].apply(lambda row: win_R(row),axis=1)


Leffen = pd.DataFrame([['arm v lef 1', game_fbf_data['genesis 5_armada VS leffen_1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS leffen_1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS leffen_1']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS leffen_1']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS leffen_1']['Win_R'][0]]],columns=['game','avg_kill_pct','avg_death_pct','Win'])

Leffen.loc[1] = ['arm v lef 2', game_fbf_data['genesis 5_armada VS leffen_2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS leffen_2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS leffen_2']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS leffen_2']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS leffen_2']['Win_R'][0]]

Leffen.loc[2] = ['arm v lef 3', game_fbf_data['genesis 5_armada VS leffen_3']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS leffen_3']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS leffen_3']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS leffen_3']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS leffen_3']['Win_R'][0]]

Leffen.loc[3] = ['arm v lef 4', game_fbf_data['genesis 5_armada VS leffen_4']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS leffen_4']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS leffen_4']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS leffen_4']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS leffen_4']['Win_R'][0]]

Leffen.loc[4] = ['lef v hbox 1', game_fbf_data['genesis 5_leffen VS hbox_1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_1']['death_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_1']['detect_death_L'].sum(),game_fbf_data['genesis 5_leffen VS hbox_1']['Win_L'][0]]

Leffen.loc[5] = ['lef v hbox 2', game_fbf_data['genesis 5_leffen VS hbox_2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_2']['death_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_2']['detect_death_L'].sum(),game_fbf_data['genesis 5_leffen VS hbox_2']['Win_L'][0]]

Leffen.loc[6] = ['lef v hbox 3', game_fbf_data['genesis 5_leffen VS hbox_3']['kill_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_3']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_3']['death_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_3']['detect_death_L'].sum(),game_fbf_data['genesis 5_leffen VS hbox_3']['Win_L'][0]]

Leffen.loc[7] = ['lef v hbox 4', game_fbf_data['genesis 5_leffen VS hbox_4']['kill_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_4']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_4']['death_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_4']['detect_death_L'].sum(),game_fbf_data['genesis 5_leffen VS hbox_4']['Win_L'][0]]

Leffen.loc[8] = ['lef v hbox 5', game_fbf_data['genesis 5_leffen VS hbox_5']['kill_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_5']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_5']['death_pct_L'].sum()/game_fbf_data['genesis 5_leffen VS hbox_5']['detect_death_L'].sum(),game_fbf_data['genesis 5_leffen VS hbox_5']['Win_L'][0]]

Leffen['avgd_stock_efficiency'] = Leffen.apply(lambda row: stockeff(row),axis=1)

print(Leffen)

game_fbf_data['genesis 5_plup VS hbox_1_WF']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_plup VS hbox_1_WF'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_plup VS hbox_1_WF'].R_Player_stocks))
game_fbf_data['genesis 5_plup VS hbox_1_WF']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_plup VS hbox_1_WF'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_plup VS hbox_1_WF'].L_Player_stocks))
game_fbf_data['genesis 5_plup VS hbox_1_WF']['death_pct_L'] = game_fbf_data['genesis 5_plup VS hbox_1_WF'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_1_WF']['death_pct_R'] = game_fbf_data['genesis 5_plup VS hbox_1_WF'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_1_WF']['kill_pct_L'] = game_fbf_data['genesis 5_plup VS hbox_1_WF'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_1_WF']['kill_pct_R'] = game_fbf_data['genesis 5_plup VS hbox_1_WF'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_1_WF']['Win_L'] = game_fbf_data['genesis 5_plup VS hbox_1_WF'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_1_WF']['Win_R'] = game_fbf_data['genesis 5_plup VS hbox_1_WF'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_plup VS hbox_2_WF']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_plup VS hbox_2_WF'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_plup VS hbox_2_WF'].R_Player_stocks))
game_fbf_data['genesis 5_plup VS hbox_2_WF']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_plup VS hbox_2_WF'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_plup VS hbox_2_WF'].L_Player_stocks))
game_fbf_data['genesis 5_plup VS hbox_2_WF']['death_pct_L'] = game_fbf_data['genesis 5_plup VS hbox_2_WF'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_2_WF']['death_pct_R'] = game_fbf_data['genesis 5_plup VS hbox_2_WF'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_2_WF']['kill_pct_L'] = game_fbf_data['genesis 5_plup VS hbox_2_WF'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_2_WF']['kill_pct_R'] = game_fbf_data['genesis 5_plup VS hbox_2_WF'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_2_WF']['Win_L'] = game_fbf_data['genesis 5_plup VS hbox_2_WF'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_2_WF']['Win_R'] = game_fbf_data['genesis 5_plup VS hbox_2_WF'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_plup VS hbox_3_WF']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_plup VS hbox_3_WF'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_plup VS hbox_3_WF'].R_Player_stocks))
game_fbf_data['genesis 5_plup VS hbox_3_WF']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_plup VS hbox_3_WF'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_plup VS hbox_3_WF'].L_Player_stocks))
game_fbf_data['genesis 5_plup VS hbox_3_WF']['death_pct_L'] = game_fbf_data['genesis 5_plup VS hbox_3_WF'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_3_WF']['death_pct_R'] = game_fbf_data['genesis 5_plup VS hbox_3_WF'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_3_WF']['kill_pct_L'] = game_fbf_data['genesis 5_plup VS hbox_3_WF'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_3_WF']['kill_pct_R'] = game_fbf_data['genesis 5_plup VS hbox_3_WF'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_3_WF']['Win_L'] = game_fbf_data['genesis 5_plup VS hbox_3_WF'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_3_WF']['Win_R'] = game_fbf_data['genesis 5_plup VS hbox_3_WF'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_plup VS hbox_4_WF']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_plup VS hbox_4_WF'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_plup VS hbox_4_WF'].R_Player_stocks))
game_fbf_data['genesis 5_plup VS hbox_4_WF']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_plup VS hbox_4_WF'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_plup VS hbox_4_WF'].L_Player_stocks))
game_fbf_data['genesis 5_plup VS hbox_4_WF']['death_pct_L'] = game_fbf_data['genesis 5_plup VS hbox_4_WF'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_4_WF']['death_pct_R'] = game_fbf_data['genesis 5_plup VS hbox_4_WF'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_4_WF']['kill_pct_L'] = game_fbf_data['genesis 5_plup VS hbox_4_WF'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_4_WF']['kill_pct_R'] = game_fbf_data['genesis 5_plup VS hbox_4_WF'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_4_WF']['Win_L'] = game_fbf_data['genesis 5_plup VS hbox_4_WF'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_plup VS hbox_4_WF']['Win_R'] = game_fbf_data['genesis 5_plup VS hbox_4_WF'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_hbox VS plup_1_GF1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_1_GF1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_1_GF1'].R_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_1_GF1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_1_GF1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_1_GF1'].L_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_1_GF1']['death_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_1_GF1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF1']['death_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_1_GF1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF1']['kill_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_1_GF1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF1']['kill_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_1_GF1'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF1']['Win_L'] = game_fbf_data['genesis 5_hbox VS plup_1_GF1'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF1']['Win_R'] = game_fbf_data['genesis 5_hbox VS plup_1_GF1'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_hbox VS plup_2_GF1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_2_GF1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_2_GF1'].R_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_2_GF1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_2_GF1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_2_GF1'].L_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_2_GF1']['death_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_2_GF1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF1']['death_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_2_GF1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF1']['kill_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_2_GF1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF1']['kill_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_2_GF1'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF1']['Win_L'] = game_fbf_data['genesis 5_hbox VS plup_2_GF1'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF1']['Win_R'] = game_fbf_data['genesis 5_hbox VS plup_2_GF1'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_hbox VS plup_3_GF1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_3_GF1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_3_GF1'].R_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_3_GF1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_3_GF1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_3_GF1'].L_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_3_GF1']['death_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_3_GF1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF1']['death_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_3_GF1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF1']['kill_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_3_GF1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF1']['kill_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_3_GF1'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF1']['Win_L'] = game_fbf_data['genesis 5_hbox VS plup_3_GF1'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF1']['Win_R'] = game_fbf_data['genesis 5_hbox VS plup_3_GF1'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_hbox VS plup_1_GF2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_1_GF2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_1_GF2'].R_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_1_GF2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_1_GF2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_1_GF2'].L_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_1_GF2']['death_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_1_GF2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF2']['death_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_1_GF2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF2']['kill_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_1_GF2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF2']['kill_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_1_GF2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF2']['Win_L'] = game_fbf_data['genesis 5_hbox VS plup_1_GF2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_1_GF2']['Win_R'] = game_fbf_data['genesis 5_hbox VS plup_1_GF2'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_hbox VS plup_2_GF2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_2_GF2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_2_GF2'].R_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_2_GF2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_2_GF2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_2_GF2'].L_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_2_GF2']['death_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_2_GF2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF2']['death_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_2_GF2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF2']['kill_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_2_GF2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF2']['kill_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_2_GF2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF2']['Win_L'] = game_fbf_data['genesis 5_hbox VS plup_2_GF2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_2_GF2']['Win_R'] = game_fbf_data['genesis 5_hbox VS plup_2_GF2'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_hbox VS plup_3_GF2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_3_GF2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_3_GF2'].R_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_3_GF2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_3_GF2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_3_GF2'].L_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_3_GF2']['death_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_3_GF2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF2']['death_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_3_GF2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF2']['kill_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_3_GF2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF2']['kill_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_3_GF2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF2']['Win_L'] = game_fbf_data['genesis 5_hbox VS plup_3_GF2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_3_GF2']['Win_R'] = game_fbf_data['genesis 5_hbox VS plup_3_GF2'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_hbox VS plup_4_GF2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_4_GF2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_4_GF2'].R_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_4_GF2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_4_GF2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_4_GF2'].L_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_4_GF2']['death_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_4_GF2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_4_GF2']['death_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_4_GF2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_4_GF2']['kill_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_4_GF2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_4_GF2']['kill_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_4_GF2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_4_GF2']['Win_L'] = game_fbf_data['genesis 5_hbox VS plup_4_GF2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_4_GF2']['Win_R'] = game_fbf_data['genesis 5_hbox VS plup_4_GF2'].apply(lambda row: win_R(row),axis=1)

game_fbf_data['genesis 5_hbox VS plup_5_GF2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_5_GF2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_5_GF2'].R_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_5_GF2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hbox VS plup_5_GF2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hbox VS plup_5_GF2'].L_Player_stocks))
game_fbf_data['genesis 5_hbox VS plup_5_GF2']['death_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_5_GF2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_5_GF2']['death_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_5_GF2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_5_GF2']['kill_pct_L'] = game_fbf_data['genesis 5_hbox VS plup_5_GF2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_5_GF2']['kill_pct_R'] = game_fbf_data['genesis 5_hbox VS plup_5_GF2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_5_GF2']['Win_L'] = game_fbf_data['genesis 5_hbox VS plup_5_GF2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_hbox VS plup_5_GF2']['Win_R'] = game_fbf_data['genesis 5_hbox VS plup_5_GF2'].apply(lambda row: win_R(row),axis=1)

Plup = pd.DataFrame([['arm v plup 1', game_fbf_data['genesis 5_armada VS plup_1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS plup_1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS plup_1']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS plup_1']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS plup_1']['Win_R'][0]]],columns=['game','avg_kill_pct','avg_death_pct','Win'])

Plup.loc[1] = ['arm v plup 2', game_fbf_data['genesis 5_armada VS plup_2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS plup_2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS plup_2']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS plup_2']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS plup_2']['Win_R'][0]]

Plup.loc[2] = ['arm v plup 3', game_fbf_data['genesis 5_armada VS plup_3']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS plup_3']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS plup_3']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS plup_3']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS plup_3']['Win_R'][0]]

Plup.loc[3] = ['plup v hbox 1 wf', game_fbf_data['genesis 5_plup VS hbox_1_WF']['kill_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_1_WF']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_plup VS hbox_1_WF']['death_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_1_WF']['detect_death_L'].sum(),game_fbf_data['genesis 5_plup VS hbox_1_WF']['Win_L'][0]]

Plup.loc[4] = ['plup v hbox 2 wf', game_fbf_data['genesis 5_plup VS hbox_2_WF']['kill_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_2_WF']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_plup VS hbox_2_WF']['death_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_2_WF']['detect_death_L'].sum(),game_fbf_data['genesis 5_plup VS hbox_2_WF']['Win_L'][0]]

Plup.loc[5] = ['plup v hbox 3 wf', game_fbf_data['genesis 5_plup VS hbox_3_WF']['kill_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_3_WF']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_plup VS hbox_3_WF']['death_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_3_WF']['detect_death_L'].sum(),game_fbf_data['genesis 5_plup VS hbox_3_WF']['Win_L'][0]]

Plup.loc[6] = ['plup v hbox 4 wf', game_fbf_data['genesis 5_plup VS hbox_4_WF']['kill_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_4_WF']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_plup VS hbox_4_WF']['death_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_4_WF']['detect_death_L'].sum(),game_fbf_data['genesis 5_plup VS hbox_4_WF']['Win_L'][0]]

Plup.loc[7] = ['hbox v plup 1 gf1', game_fbf_data['genesis 5_hbox VS plup_1_GF1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_1_GF1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_1_GF1']['death_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_1_GF1']['detect_death_R'].sum(),game_fbf_data['genesis 5_hbox VS plup_1_GF1']['Win_R'][0]]

Plup.loc[8] = ['hbox v plup 2 gf1', game_fbf_data['genesis 5_hbox VS plup_2_GF1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_2_GF1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_2_GF1']['death_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_2_GF1']['detect_death_R'].sum(),game_fbf_data['genesis 5_hbox VS plup_2_GF1']['Win_R'][0]]

Plup.loc[9] = ['hbox v plup 3 gf1', game_fbf_data['genesis 5_hbox VS plup_3_GF1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_3_GF1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_3_GF1']['death_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_3_GF1']['detect_death_R'].sum(),game_fbf_data['genesis 5_hbox VS plup_3_GF1']['Win_R'][0]]

Plup.loc[10] = ['hbox v plup 1 gf2', game_fbf_data['genesis 5_hbox VS plup_1_GF2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_1_GF2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_1_GF2']['death_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_1_GF2']['detect_death_R'].sum(),game_fbf_data['genesis 5_hbox VS plup_1_GF2']['Win_R'][0]]

Plup.loc[11] = ['hbox v plup 2 gf2', game_fbf_data['genesis 5_hbox VS plup_2_GF2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_2_GF2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_2_GF1']['death_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_2_GF2']['detect_death_R'].sum(),game_fbf_data['genesis 5_hbox VS plup_2_GF2']['Win_R'][0]]

Plup.loc[12] = ['hbox v plup 3 gf2', game_fbf_data['genesis 5_hbox VS plup_3_GF2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_3_GF2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_3_GF2']['death_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_3_GF2']['detect_death_R'].sum(),game_fbf_data['genesis 5_hbox VS plup_3_GF2']['Win_R'][0]]

Plup.loc[13] = ['hbox v plup 4 gf2', game_fbf_data['genesis 5_hbox VS plup_4_GF2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_4_GF2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_4_GF2']['death_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_4_GF2']['detect_death_R'].sum(),game_fbf_data['genesis 5_hbox VS plup_4_GF2']['Win_R'][0]]

Plup.loc[14] = ['hbox v plup 5 gf2', game_fbf_data['genesis 5_hbox VS plup_5_GF2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_5_GF2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_2_GF2']['death_pct_R'].sum()/game_fbf_data['genesis 5_hbox VS plup_5_GF2']['detect_death_R'].sum(),game_fbf_data['genesis 5_hbox VS plup_5_GF2']['Win_R'][0]]

Plup['avgd_stock_efficiency'] = Plup.apply(lambda row: stockeff(row),axis=1)

print(Plup)

Hbox = pd.DataFrame([['lef v hbox 1', game_fbf_data['genesis 5_leffen VS hbox_1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_1']['death_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_1']['detect_death_R'].sum(),game_fbf_data['genesis 5_leffen VS hbox_1']['Win_R'][0]]],columns=['game','avg_kill_pct','avg_death_pct','Win'])

Hbox.loc[1] = ['lef v hbox 2', game_fbf_data['genesis 5_leffen VS hbox_2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_2']['death_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_2']['detect_death_R'].sum(),game_fbf_data['genesis 5_leffen VS hbox_2']['Win_R'][0]]

Hbox.loc[2] = ['lef v hbox 3', game_fbf_data['genesis 5_leffen VS hbox_3']['kill_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_3']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_3']['death_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_3']['detect_death_R'].sum(),game_fbf_data['genesis 5_leffen VS hbox_3']['Win_R'][0]]

Hbox.loc[3] = ['lef v hbox 4', game_fbf_data['genesis 5_leffen VS hbox_4']['kill_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_4']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_4']['death_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_4']['detect_death_R'].sum(),game_fbf_data['genesis 5_leffen VS hbox_4']['Win_R'][0]]

Hbox.loc[4] = ['lef v hbox 5', game_fbf_data['genesis 5_leffen VS hbox_5']['kill_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_5']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_leffen VS hbox_5']['death_pct_R'].sum()/game_fbf_data['genesis 5_leffen VS hbox_5']['detect_death_R'].sum(),game_fbf_data['genesis 5_leffen VS hbox_5']['Win_R'][0]]

Hbox.loc[5] = ['plup v hbox 1 wf', game_fbf_data['genesis 5_plup VS hbox_1_WF']['kill_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_1_WF']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_plup VS hbox_1_WF']['death_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_1_WF']['detect_death_L'].sum(),game_fbf_data['genesis 5_plup VS hbox_1_WF']['Win_L'][0]]

Hbox.loc[6] = ['plup v hbox 2 wf', game_fbf_data['genesis 5_plup VS hbox_2_WF']['kill_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_2_WF']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_plup VS hbox_2_WF']['death_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_2_WF']['detect_death_L'].sum(),game_fbf_data['genesis 5_plup VS hbox_2_WF']['Win_L'][0]]

Hbox.loc[7] = ['plup v hbox 3 wf', game_fbf_data['genesis 5_plup VS hbox_3_WF']['kill_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_3_WF']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_plup VS hbox_3_WF']['death_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_3_WF']['detect_death_L'].sum(),game_fbf_data['genesis 5_plup VS hbox_3_WF']['Win_L'][0]]

Hbox.loc[8] = ['plup v hbox 4 wf', game_fbf_data['genesis 5_plup VS hbox_4_WF']['kill_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_4_WF']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_plup VS hbox_4_WF']['death_pct_L'].sum()/game_fbf_data['genesis 5_plup VS hbox_4_WF']['detect_death_L'].sum(),game_fbf_data['genesis 5_plup VS hbox_4_WF']['Win_L'][0]]

Hbox.loc[9] = ['hbox v plup 1 gf1', game_fbf_data['genesis 5_hbox VS plup_1_GF1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_1_GF1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_1_GF1']['death_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_1_GF1']['detect_death_L'].sum(),game_fbf_data['genesis 5_hbox VS plup_1_GF1']['Win_L'][0]]

Hbox.loc[10] = ['hbox v plup 2 gf1', game_fbf_data['genesis 5_hbox VS plup_2_GF1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_2_GF1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_2_GF1']['death_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_2_GF1']['detect_death_L'].sum(),game_fbf_data['genesis 5_hbox VS plup_2_GF1']['Win_L'][0]]

Hbox.loc[11] = ['hbox v plup 3 gf1', game_fbf_data['genesis 5_hbox VS plup_3_GF1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_3_GF1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_3_GF1']['death_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_3_GF1']['detect_death_L'].sum(),game_fbf_data['genesis 5_hbox VS plup_3_GF1']['Win_L'][0]]

Hbox.loc[12] = ['hbox v plup 1 gf2', game_fbf_data['genesis 5_hbox VS plup_1_GF2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_1_GF2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_1_GF2']['death_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_1_GF2']['detect_death_L'].sum(),game_fbf_data['genesis 5_hbox VS plup_1_GF2']['Win_L'][0]]

Hbox.loc[13] = ['hbox v plup 2 gf2', game_fbf_data['genesis 5_hbox VS plup_2_GF2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_2_GF2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_2_GF2']['death_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_2_GF2']['detect_death_L'].sum(),game_fbf_data['genesis 5_hbox VS plup_2_GF2']['Win_L'][0]]

Hbox.loc[14] = ['hbox v plup 3 gf2', game_fbf_data['genesis 5_hbox VS plup_3_GF2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_3_GF2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_3_GF2']['death_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_3_GF2']['detect_death_L'].sum(),game_fbf_data['genesis 5_hbox VS plup_3_GF2']['Win_L'][0]]

Hbox.loc[15] = ['hbox v plup 4 gf2', game_fbf_data['genesis 5_hbox VS plup_4_GF2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_4_GF2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_4_GF2']['death_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_4_GF2']['detect_death_L'].sum(),game_fbf_data['genesis 5_hbox VS plup_4_GF2']['Win_L'][0]]

Hbox.loc[16] = ['hbox v plup 5 gf2', game_fbf_data['genesis 5_hbox VS plup_5_GF2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_5_GF2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hbox VS plup_5_GF2']['death_pct_L'].sum()/game_fbf_data['genesis 5_hbox VS plup_5_GF2']['detect_death_L'].sum(),game_fbf_data['genesis 5_hbox VS plup_5_GF2']['Win_L'][0]]

Hbox['avgd_stock_efficiency'] = Hbox.apply(lambda row: stockeff(row),axis=1)

print(Hbox)

game_fbf_data['genesis 5_hugs VS lucky_1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hugs VS lucky_1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hugs VS lucky_1'].R_Player_stocks))
game_fbf_data['genesis 5_hugs VS lucky_1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hugs VS lucky_1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hugs VS lucky_1'].L_Player_stocks))
game_fbf_data['genesis 5_hugs VS lucky_1']['death_pct_L'] = game_fbf_data['genesis 5_hugs VS lucky_1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_1']['death_pct_R'] = game_fbf_data['genesis 5_hugs VS lucky_1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_1']['kill_pct_L'] = game_fbf_data['genesis 5_hugs VS lucky_1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_1']['kill_pct_R'] = game_fbf_data['genesis 5_hugs VS lucky_1'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_1']['Win_R'] = game_fbf_data['genesis 5_hugs VS lucky_1'].apply(lambda row: win_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_1']['Win_L'] = game_fbf_data['genesis 5_hugs VS lucky_1'].apply(lambda row: win_L(row),axis=1)

game_fbf_data['genesis 5_hugs VS lucky_2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hugs VS lucky_2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hugs VS lucky_2'].R_Player_stocks))
game_fbf_data['genesis 5_hugs VS lucky_2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hugs VS lucky_2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hugs VS lucky_2'].L_Player_stocks))
game_fbf_data['genesis 5_hugs VS lucky_2']['death_pct_L'] = game_fbf_data['genesis 5_hugs VS lucky_2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_2']['death_pct_R'] = game_fbf_data['genesis 5_hugs VS lucky_2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_2']['kill_pct_L'] = game_fbf_data['genesis 5_hugs VS lucky_2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_2']['kill_pct_R'] = game_fbf_data['genesis 5_hugs VS lucky_2'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_2']['Win_R'] = game_fbf_data['genesis 5_hugs VS lucky_2'].apply(lambda row: win_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_2']['Win_L'] = game_fbf_data['genesis 5_hugs VS lucky_2'].apply(lambda row: win_L(row),axis=1)


game_fbf_data['genesis 5_hugs VS lucky_3']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_hugs VS lucky_3'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hugs VS lucky_3'].R_Player_stocks))
game_fbf_data['genesis 5_hugs VS lucky_3']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_hugs VS lucky_3'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_hugs VS lucky_3'].L_Player_stocks))
game_fbf_data['genesis 5_hugs VS lucky_3']['death_pct_L'] = game_fbf_data['genesis 5_hugs VS lucky_3'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_3']['death_pct_R'] = game_fbf_data['genesis 5_hugs VS lucky_3'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_3']['kill_pct_L'] = game_fbf_data['genesis 5_hugs VS lucky_3'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_3']['kill_pct_R'] = game_fbf_data['genesis 5_hugs VS lucky_3'].apply(lambda row: killpct_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_3']['Win_R'] = game_fbf_data['genesis 5_hugs VS lucky_3'].apply(lambda row: win_R(row),axis=1)
game_fbf_data['genesis 5_hugs VS lucky_3']['Win_L'] = game_fbf_data['genesis 5_hugs VS lucky_3'].apply(lambda row: win_L(row),axis=1)


Lucky = pd.DataFrame([['arm v luck 1', game_fbf_data['genesis 5_armada VS lucky_1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS lucky_1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS lucky_1']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS lucky_1']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS lucky_1']['Win_R'][0]]],columns=['game','avg_kill_pct','avg_death_pct','Win'])

Lucky.loc[1] = ['arm v luck 2', game_fbf_data['genesis 5_armada VS lucky_2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS lucky_2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS lucky_2']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS lucky_2']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS lucky_2']['Win_R'][0]]

Lucky.loc[2] = ['arm v luck 3', game_fbf_data['genesis 5_armada VS lucky_3']['kill_pct_R'].sum()/game_fbf_data['genesis 5_armada VS lucky_3']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_armada VS lucky_3']['death_pct_R'].sum()/game_fbf_data['genesis 5_armada VS lucky_3']['detect_death_R'].sum(),game_fbf_data['genesis 5_armada VS lucky_3']['Win_R'][0]]

Lucky.loc[3] = ['hugs v luck 1', game_fbf_data['genesis 5_hugs VS lucky_1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hugs VS lucky_1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hugs VS lucky_1']['death_pct_R'].sum()/game_fbf_data['genesis 5_hugs VS lucky_1']['detect_death_R'].sum(),game_fbf_data['genesis 5_hugs VS lucky_1']['Win_R'][0]]

Lucky.loc[4] = ['hugs v luck 2', game_fbf_data['genesis 5_hugs VS lucky_2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hugs VS lucky_2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hugs VS lucky_2']['death_pct_R'].sum()/game_fbf_data['genesis 5_hugs VS lucky_2']['detect_death_R'].sum(),game_fbf_data['genesis 5_hugs VS lucky_2']['Win_R'][0]]

Lucky.loc[5] = ['hugs v luck 3', game_fbf_data['genesis 5_hugs VS lucky_3']['kill_pct_R'].sum()/game_fbf_data['genesis 5_hugs VS lucky_3']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_hugs VS lucky_3']['death_pct_R'].sum()/game_fbf_data['genesis 5_hugs VS lucky_3']['detect_death_R'].sum(),game_fbf_data['genesis 5_hugs VS lucky_3']['Win_R'][0]]

Lucky['avgd_stock_efficiency'] = Lucky.apply(lambda row: stockeff(row),axis=1)

print(Lucky)

Hugs = pd.DataFrame([['hugs v luck 1', game_fbf_data['genesis 5_hugs VS lucky_1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hugs VS lucky_1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hugs VS lucky_1']['death_pct_L'].sum()/game_fbf_data['genesis 5_hugs VS lucky_1']['detect_death_L'].sum(),game_fbf_data['genesis 5_hugs VS lucky_1']['Win_L'][0]]],columns=['game','avg_kill_pct','avg_death_pct','Win'])

Hugs.loc[1] = ['hugs v luck 2', game_fbf_data['genesis 5_hugs VS lucky_2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hugs VS lucky_2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hugs VS lucky_2']['death_pct_L'].sum()/game_fbf_data['genesis 5_hugs VS lucky_2']['detect_death_L'].sum(),game_fbf_data['genesis 5_hugs VS lucky_2']['Win_L'][0]]

Hugs.loc[5] = ['hugs v luck 3', game_fbf_data['genesis 5_hugs VS lucky_3']['kill_pct_L'].sum()/game_fbf_data['genesis 5_hugs VS lucky_3']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_hugs VS lucky_3']['death_pct_L'].sum()/game_fbf_data['genesis 5_hugs VS lucky_3']['detect_death_L'].sum(),game_fbf_data['genesis 5_hugs VS lucky_3']['Win_L'][0]]

Hugs['avgd_stock_efficiency'] = Hugs.apply(lambda row: stockeff(row),axis=1)

print(Hugs)

game_fbf_data['genesis 5_axe VS mango_1']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_1'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_1'].R_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_1']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_1'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_1'].L_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_1']['death_pct_L'] = game_fbf_data['genesis 5_axe VS mango_1'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_1']['death_pct_R'] = game_fbf_data['genesis 5_axe VS mango_1'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_1']['kill_pct_L'] = game_fbf_data['genesis 5_axe VS mango_1'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_1']['kill_pct_R'] = game_fbf_data['genesis 5_axe VS mango_1'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_1']['Win_R'] = game_fbf_data['genesis 5_axe VS mango_1'].apply(lambda row: win_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_1']['Win_L'] = game_fbf_data['genesis 5_axe VS mango_1'].apply(lambda row: win_L(row),axis=1)

game_fbf_data['genesis 5_axe VS mango_2']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_2'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_2'].R_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_2']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_2'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_2'].L_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_2']['death_pct_L'] = game_fbf_data['genesis 5_axe VS mango_2'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_2']['death_pct_R'] = game_fbf_data['genesis 5_axe VS mango_2'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_2']['kill_pct_L'] = game_fbf_data['genesis 5_axe VS mango_2'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_2']['kill_pct_R'] = game_fbf_data['genesis 5_axe VS mango_2'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_2']['Win_R'] = game_fbf_data['genesis 5_axe VS mango_2'].apply(lambda row: win_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_2']['Win_L'] = game_fbf_data['genesis 5_axe VS mango_2'].apply(lambda row: win_L(row),axis=1)

game_fbf_data['genesis 5_axe VS mango_3']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_3'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_3'].R_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_3']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_3'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_3'].L_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_3']['death_pct_L'] = game_fbf_data['genesis 5_axe VS mango_3'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_3']['death_pct_R'] = game_fbf_data['genesis 5_axe VS mango_3'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_3']['kill_pct_L'] = game_fbf_data['genesis 5_axe VS mango_3'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_3']['kill_pct_R'] = game_fbf_data['genesis 5_axe VS mango_3'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_3']['Win_R'] = game_fbf_data['genesis 5_axe VS mango_3'].apply(lambda row: win_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_3']['Win_L'] = game_fbf_data['genesis 5_axe VS mango_3'].apply(lambda row: win_L(row),axis=1)

game_fbf_data['genesis 5_axe VS mango_4']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_4'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_4'].R_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_4']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_4'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_4'].L_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_4']['death_pct_L'] = game_fbf_data['genesis 5_axe VS mango_4'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_4']['death_pct_R'] = game_fbf_data['genesis 5_axe VS mango_4'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_4']['kill_pct_L'] = game_fbf_data['genesis 5_axe VS mango_4'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_4']['kill_pct_R'] = game_fbf_data['genesis 5_axe VS mango_4'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_4']['Win_R'] = game_fbf_data['genesis 5_axe VS mango_4'].apply(lambda row: win_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_4']['Win_L'] = game_fbf_data['genesis 5_axe VS mango_4'].apply(lambda row: win_L(row),axis=1)

game_fbf_data['genesis 5_axe VS mango_5']['detect_death_R'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_5'].R_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_5'].R_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_5']['detect_death_L'] = detectdeath(game_fbf_data['genesis 5_axe VS mango_5'].L_Player_stocks,makedeatharray(game_fbf_data['genesis 5_axe VS mango_5'].L_Player_stocks))
game_fbf_data['genesis 5_axe VS mango_5']['death_pct_L'] = game_fbf_data['genesis 5_axe VS mango_5'].apply(lambda row: deathpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_5']['death_pct_R'] = game_fbf_data['genesis 5_axe VS mango_5'].apply(lambda row: deathpct_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_5']['kill_pct_L'] = game_fbf_data['genesis 5_axe VS mango_5'].apply(lambda row: killpct_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_5']['kill_pct_R'] = game_fbf_data['genesis 5_axe VS mango_5'].apply(lambda row: win_L(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_5']['Win_R'] = game_fbf_data['genesis 5_axe VS mango_5'].apply(lambda row: win_R(row),axis=1)
game_fbf_data['genesis 5_axe VS mango_5']['Win_L'] = game_fbf_data['genesis 5_axe VS mango_5'].apply(lambda row: win_L(row),axis=1)

Axe = pd.DataFrame([['axe v mang 1', game_fbf_data['genesis 5_axe VS mango_1']['kill_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_1']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_1']['death_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_1']['detect_death_L'].sum(),game_fbf_data['genesis 5_axe VS mango_1']['Win_L'][0]]],columns=['game','avg_kill_pct','avg_death_pct','Win'])

Axe.loc[1] = ['axe v mang 2', game_fbf_data['genesis 5_axe VS mango_2']['kill_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_2']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_2']['death_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_2']['detect_death_L'].sum(),game_fbf_data['genesis 5_axe VS mango_2']['Win_L'][0]]

Axe.loc[2] = ['axe v mang 3', game_fbf_data['genesis 5_axe VS mango_3']['kill_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_3']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_3']['death_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_3']['detect_death_L'].sum(),game_fbf_data['genesis 5_axe VS mango_3']['Win_L'][0]]

Axe.loc[3] = ['axe v mang 4', game_fbf_data['genesis 5_axe VS mango_4']['kill_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_4']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_4']['death_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_4']['detect_death_L'].sum(),game_fbf_data['genesis 5_axe VS mango_4']['Win_L'][0]]

Axe.loc[4] = ['axe v mang 5', game_fbf_data['genesis 5_axe VS mango_5']['kill_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_5']['detect_death_R'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_5']['death_pct_L'].sum()/game_fbf_data['genesis 5_axe VS mango_5']['detect_death_L'].sum(),game_fbf_data['genesis 5_axe VS mango_5']['Win_L'][0]]

Axe['avgd_stock_efficiency'] = Axe.apply(lambda row: stockeff(row),axis=1)

print(Axe)

Mango = pd.DataFrame([['axe v mang 1', game_fbf_data['genesis 5_axe VS mango_1']['kill_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_1']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_1']['death_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_1']['detect_death_R'].sum(),game_fbf_data['genesis 5_axe VS mango_1']['Win_R'][0]]],columns=['game','avg_kill_pct','avg_death_pct','Win'])

Mango.loc[1] = ['axe v mang 2', game_fbf_data['genesis 5_axe VS mango_2']['kill_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_2']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_2']['death_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_2']['detect_death_R'].sum(),game_fbf_data['genesis 5_axe VS mango_2']['Win_R'][0]]

Mango.loc[2] = ['axe v mang 3', game_fbf_data['genesis 5_axe VS mango_3']['kill_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_3']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_3']['death_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_3']['detect_death_R'].sum(),game_fbf_data['genesis 5_axe VS mango_3']['Win_R'][0]]

Mango.loc[3] = ['axe v mang 4', game_fbf_data['genesis 5_axe VS mango_4']['kill_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_4']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_4']['death_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_4']['detect_death_R'].sum(),game_fbf_data['genesis 5_axe VS mango_4']['Win_R'][0]]

Mango.loc[4] = ['axe v mang 5', game_fbf_data['genesis 5_axe VS mango_5']['kill_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_5']['detect_death_L'].sum(),\
     game_fbf_data['genesis 5_axe VS mango_5']['death_pct_R'].sum()/game_fbf_data['genesis 5_axe VS mango_5']['detect_death_R'].sum(),game_fbf_data['genesis 5_axe VS mango_5']['Win_R'][0]]

Mango['avgd_stock_efficiency'] = Mango.apply(lambda row: stockeff(row),axis=1)

print(Mango)

AttributeError: 'dict' object has no attribute 'R_Player_stocks'